# Objective:

As input to the system, take the live feed from the webcam and use pose estimation to map out a small dance tutorial.

# Approach:
- We will take a pretrained **openpose estimation model** to prdict the **18 keypoints** on a human body.
- We take openpose model for tensorflow by Ildoo Kim
  - GitHub Repo Link: https://github.com/ildoonet/tf-pose-estimation
<br>**[!] Note**: Some how I found issues with this repo to work with tensorflow 2.0 and followed a modified repo of his by Gunjan Seth.<br>
GitHub Repo Link: https://github.com/gsethi2409/tf-pose-estimation
<br>Medium Blog by Gunjan Seth: https://medium.com/@gsethi2409/pose-estimation-with-tensorflow-2-0-a51162c095ba
- The keypoints of the dancer are obtained and stored in a array list.
- These keypoints are **normalized**.
- The user feed is taken and the keypoints are detected.
- The keypoints are normalized and the **cosine similarity** is found between the user keypoints and the array of dancer keypoints.
- The minimum similarity score is **compared with the threshold** and then it displays is the user steps are correct or not for the given dancer moves.

# Constraints To Look For:
1. The model should be fast for prediction. Latency should be avoided.
2. Predictions should be accurate and the steps should be close enough with  the dancer.


## Import the Necessary Libraries

In [1]:
import sys
import time
import logging
import numpy as np
import cv2

import numpy as np
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings('ignore')


## Model and TfPose Estimator
We initialize the pretrained model with the required parameters as seen below.

In [2]:
camera = 0
resize = '432x368'     # resize images before they are processed
resize_out_ratio = 4.0 # resize heatmaps before they are post-processed
model='mobilenet_v2_large'
show_process = False
tensorrt = False       # for tensorrt process

In [3]:
w, h = model_wh(resize)
if w > 0 and h > 0:
    e = TfPoseEstimator(get_graph_path(model), target_size=(w, h), trt_bool=False)
else:
    e = TfPoseEstimator(get_graph_path(model), target_size=(432, 368), trt_bool=False)
print('********* Model Ready *************')

[2020-08-17 07:40:15,769] [TfPoseEstimator] [INFO] loading graph from F:\Position Estimation\tf-pose-estimation\models\graph/mobilenet_v2_large/graph_opt.pb(default size=432x368)
2020-08-17 07:40:15,769 INFO loading graph from F:\Position Estimation\tf-pose-estimation\models\graph/mobilenet_v2_large/graph_opt.pb(default size=432x368)


TfPoseEstimator/image
TfPoseEstimator/MobilenetV2/Conv/BatchNorm/Const
TfPoseEstimator/MobilenetV2/Conv/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv/depthwise/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv/depthwise/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv/project/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv/project/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/expand/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/expand/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_1/project/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_1/project/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_2/expand/BatchNorm/Const
TfPoseEstimator/MobilenetV2/expanded_conv_2/expand/BatchNorm/Const_1
TfPoseEstimator/MobilenetV2/expanded_conv_2/depthwise/

TfPoseEstimator/Openpose/MConv_Stage4_L2_2_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage4_L2_3_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage4_L2_4_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage4_L2_5_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_concat/axis
TfPoseEstimator/Openpose/MConv_Stage5_L1_1_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L1_2_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L1_3_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L1_5_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L2_1_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L2_2_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L2_3_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L2_4_pointwise/BatchNorm/Const
TfPoseEstimator/Openpose/MConv_Stage5_L2_5_pointwise/

TfPoseEstimator/MobilenetV2/expanded_conv_3/expand/BatchNorm/gamma/read/_58__cf__58
TfPoseEstimator/MobilenetV2/expanded_conv_3/expand/weights/read/_59__cf__59
TfPoseEstimator/MobilenetV2/expanded_conv_3/expand/Conv2D
TfPoseEstimator/MobilenetV2/expanded_conv_3/expand/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_3/expand/Relu6
TfPoseEstimator/MobilenetV2/expanded_conv_3/depthwise/depthwise
TfPoseEstimator/MobilenetV2/expanded_conv_3/depthwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_3/depthwise/Relu6
TfPoseEstimator/MobilenetV2/expanded_conv_3/project/BatchNorm/beta/read/_60__cf__60
TfPoseEstimator/MobilenetV2/expanded_conv_3/project/BatchNorm/gamma/read/_61__cf__61
TfPoseEstimator/MobilenetV2/expanded_conv_3/project/weights/read/_62__cf__62
TfPoseEstimator/MobilenetV2/expanded_conv_3/project/Conv2D
TfPoseEstimator/MobilenetV2/expanded_conv_3/project/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_4/depthwise/BatchNorm/

TfPoseEstimator/MobilenetV2/expanded_conv_9/expand/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_9/expand/Relu6
TfPoseEstimator/MobilenetV2/expanded_conv_9/depthwise/depthwise
TfPoseEstimator/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_9/depthwise/Relu6
TfPoseEstimator/MobilenetV2/expanded_conv_9/project/BatchNorm/beta/read/_114__cf__114
TfPoseEstimator/MobilenetV2/expanded_conv_9/project/BatchNorm/gamma/read/_115__cf__115
TfPoseEstimator/MobilenetV2/expanded_conv_9/project/weights/read/_116__cf__116
TfPoseEstimator/MobilenetV2/expanded_conv_9/project/Conv2D
TfPoseEstimator/MobilenetV2/expanded_conv_9/project/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_9/add
TfPoseEstimator/MobilenetV2/expanded_conv_10/expand/Conv2D
TfPoseEstimator/MobilenetV2/expanded_conv_10/expand/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV2/expanded_conv_10/expand/Relu6
TfPoseEstimator/MobilenetV2/expand

...

TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/BatchNorm/beta/read/_273__cf__273
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/BatchNorm/moving_mean/read/_274__cf__274
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/BatchNorm/moving_variance/read/_275__cf__275
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/weights/read/_276__cf__276
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/Conv2D
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/Openpose/MConv_Stage4_L1_2_pointwise/Relu
TfPoseEstimator/Openpose/MConv_Stage4_L1_3_depthwise/depthwise_weights/read/_277__cf__277
TfPoseEstimator/Openpose/MConv_Stage4_L1_3_depthwise/depthwise
TfPoseEstimator/Openpose/MConv_Stage4_L1_3_pointwise/BatchNorm/beta/read/_278__cf__278
TfPoseEstimator/Openpose/MConv_Stage4_L1_3_pointwise/BatchNorm/moving_mean/read/_279__cf__279
TfPoseEstimator/Openpose/MConv_Stage4_L1_3_pointwise/BatchNorm/moving_variance/read/_280__cf__280
TfPoseEstimat

TfPoseEstimator/Openpose/MConv_Stage5_L1_3_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/Openpose/MConv_Stage5_L1_3_pointwise/Relu
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_depthwise/depthwise_weights/read/_332__cf__332
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_depthwise/depthwise
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/BatchNorm/beta/read/_333__cf__333
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/BatchNorm/moving_mean/read/_334__cf__334
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/BatchNorm/moving_variance/read/_335__cf__335
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/weights/read/_336__cf__336
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/Conv2D
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/Openpose/MConv_Stage5_L1_4_pointwise/Relu
TfPoseEstimator/Openpose/MConv_Stage5_L1_5_depthwise/depthwise_weights/read/_337__cf__337
TfPoseEstimator/Openpose/MConv_Stage5_L1_5_depthwise/depthwise
TfP

TfPoseEstimator/Openpose/MConv_Stage6_L1_5_pointwise/BatchNorm/moving_variance/read/_390__cf__390
TfPoseEstimator/Openpose/MConv_Stage6_L1_5_pointwise/weights/read/_391__cf__391
TfPoseEstimator/Openpose/MConv_Stage6_L1_5_pointwise/Conv2D
TfPoseEstimator/Openpose/MConv_Stage6_L1_5_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_depthwise/depthwise_weights/read/_392__cf__392
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_depthwise/depthwise
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/BatchNorm/beta/read/_393__cf__393
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/BatchNorm/moving_mean/read/_394__cf__394
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/BatchNorm/moving_variance/read/_395__cf__395
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/weights/read/_396__cf__396
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/Conv2D
TfPoseEstimator/Openpose/MConv_Stage6_L2_1_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/Openpose/MConv_Sta

2020-08-17 07:40:30,567 WARNING From F:\Anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1666: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


********* Model Ready *************


# Take position from the trainer (dancer):
- We made two functions to get all the keypoints from the trainer and store them in a dataframe and in a list.
-  The function **"dance_video_processing"** is used to predict all the keypoints from the video and return all the keypoints for the video.
- The function **"get_position"** is used to take all the keypoints that are returned from the above function, preprocess them and return the dataframe and the list of keypoints.  

In [4]:
def dance_video_processing(video_path= r'dance_video/dancer.mp4',showBG = True):
        cap = cv2.VideoCapture(video_path)
        if cap.isOpened() is False:
            print("Error opening video stream or file")
        fps_time = 0
        while True:
            ret_val, image = cap.read()
            dim = (368, 428)
            if ret_val:
                 # resize image
                image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
                humans = e.inference(image,
                                     resize_to_default=(w > 0 and h > 0),
                                     upsample_size=4.0)
                if not showBG:
                    image = np.zeros(image.shape)
                # Plotting the keypoints and lines to the image 
                image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
                npimg = np.copy(image)
                image_h, image_w = npimg.shape[:2]
                centers = {}
                keypoints_list=[]
                for human in humans:
                          # draw point
                        for i in range(common.CocoPart.Background.value):
                                if i not in human.body_parts.keys():
                                        continue

                                body_part = human.body_parts[i]
                                x_axis=int(body_part.x * image_w + 0.5)
                                y_axis=int(body_part.y * image_h + 0.5) 
                                center=[x_axis,y_axis]
                                centers[i] = center
                                keypoints_list.append(centers)
                # To display fps
                cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                # To display image
                cv2.imshow('Dancer', image)
                fps_time = time.time()
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                
            else:
                break
        #print(keypoints_list)
        cap.release()
        cv2.destroyAllWindows()
        return keypoints_list

In [5]:
def get_position(video_path= r'dance_video/dancer.mp4',showBG = True):
    keypoints_list=dance_video_processing()
    import pandas as pd
    #features=[0]*32
    features=[0]*36
    #print(features)
    keyp_list=[]
    #data=pd.Dataframe()
    #print(len(keypoints_list[i]))
    # Preprocessing of the keypoints data
    for i in range(0, len(keypoints_list)):
        k=-2
        for j in range(0,18):
            k=k+2
            try:
                if k>=36:
                    break
                #print(k)
                #print(keypoints_list[i][j])
                features[k]=keypoints_list[i][j][0]
                features[k+1]=keypoints_list[i][j][1]
            except:
                features[k]=0
                features[k+1]=0
        #print(features)
        keyp_list.append(features)
    #print(keyp_list)
    # Getting all the feature column names for intialization of our dataframe.
    column_names=[]
    for i in range(36):
        column_names.append(str(i))
    data=pd.DataFrame(keyp_list,columns=column_names)
    return data,keyp_list

In [6]:
data,keyp_list=get_position()
data.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,199,79,199,107,165,107,145,133,121,135,...,256,305,191,72,206,72,177,74,218,74
1,199,79,199,107,165,107,145,133,121,135,...,256,305,191,72,206,72,177,74,218,74
2,199,79,199,107,165,107,145,133,121,135,...,256,305,191,72,206,72,177,74,218,74
3,199,79,199,107,165,107,145,133,121,135,...,256,305,191,72,206,72,177,74,218,74
4,199,79,199,107,165,107,145,133,121,135,...,256,305,191,72,206,72,177,74,218,74



**Observation:** 
- We can see how the keypoints data looks from the above example.
- Since they are 18 keypoints and each keypoint has **x-coordinate** and **y-coordinate** we have **36 columns** (18 x 2).

# Cosine Similarity:
Cosine Similarity function for our model to find the keypoints.

In [7]:
def findCosineSimilarity_1(source_representation, test_representation):
    import numpy as np
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

# Comparing:
Comparing the user images with keypoints of the dancer. 

In [8]:
def compare_positions(trainer_video,user_video,keyp_list, dim=(420,720)):
    cap = cv2.VideoCapture(trainer_video)
    cam = cv2.VideoCapture(user_video) 
    cam.set(3, w)
    cam.set(4, h)
    fps_time = 0 #Initializing fps to 0
    while True:
        ret_val, image_1 = cam.read()
        e_d=0
        ret_val_1,image_2=cap.read()
        if ret_val_1 and ret_val:
            # resizing the images
            image_2 = cv2.resize(image_2, dim, interpolation = cv2.INTER_AREA)
            image_1 = cv2.resize(image_1, dim, interpolation = cv2.INTER_AREA)
            dancers_1=e.inference(image_2,resize_to_default=(w > 0 and h > 0),upsample_size=4.0)
            humans_2 = e.inference(image_1, resize_to_default=(w > 0 and h > 0),upsample_size=4.0 )
            #Dancer keypoints and normalization
            transformer = Normalizer().fit(keyp_list)  
            keyp_list=transformer.transform(keyp_list)
            # Showing FPS
            cv2.putText(image_2, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Displaying the dancer feed.
            cv2.imshow('Dancer Window', image_2)
            # Getting User keypoints, normalization and comparing also plotting the keypoints and lines to the image
            image_1 = TfPoseEstimator.draw_humans(image_1, humans_2, imgcopy=False)
            npimg = np.copy(image_1)
            image_h, image_w = npimg.shape[:2]
            centers = {}
            keypoints_list=[]
            for human in humans_2:
                          # draw point
                    for i in range(common.CocoPart.Background.value):
                                if i not in human.body_parts.keys():
                                        continue

                                body_part = human.body_parts[i]
                                x_axis=int(body_part.x * image_w + 0.5)
                                y_axis=int(body_part.y * image_h + 0.5)
                                center=[x_axis,y_axis]
                                centers[i] = center
                    k=-2
                    features=[0]*36
                    for j in range(0,18):
                        k=k+2
                        try:
                            if k>=36:
                                break
                            #print(k)
                            #print(keypoints_list[i][j])
                            features[k]=centers[j][0]
                            features[k+1]=centers[j][1]
                        except:
                            features[k]=0
                            features[k+1]=0
                    features=transformer.transform([features])
                    #print(features[0])
                    min_=100 # Intializing a value to get minimum cosine similarity score from the dancer array list with the user
                    for j in keyp_list:
                        #print(j)
                        sim_score=findCosineSimilarity_1(j,features[0])
                        #print(sim_score)
                        #Getting the minimum Cosine Similarity Score
                        if min_>sim_score:
                            min_=sim_score
            # Displaying the minimum cosine score
            cv2.putText(image_1, str(min_), (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            # If the disctance is below the threshold
            if min_<0.15:
                cv2.putText(image_1, "CORRECT STEPS", (120, 700),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(image_1,  "NOT CORRECT STEPS", (80, 700),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image_1, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Display the user feed
            cv2.imshow('User Window', image_1)

            fps_time = time.time()
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cam.release()
    cap.release()
    cv2.destroyAllWindows()

##### Note:
Since I cant dance, I'll be using a video for this :P.<br> We can replce the **user_video** attribute to **0 or 1** to turn on live camera depending on the type of camera we have.
### For a wrong positions:

In [9]:
compare_positions(r'dance_video/dancer.mp4',r'dance_video/wrong_dance.mp4',keyp_list)

### For a correct positions:

In [10]:
compare_positions(r'dance_video/dancer.mp4',r'dance_video/right_dance.mp4',keyp_list)

# Conclusion:

- We have developed a pose estimation similarity pipeline to compare similarity between two poses from the given feed of videos or live cam.<br>
**Flaws:**
- This approach fails when the trainer is far or the user is near to the camera or vise-versa. This happens because there is a **scale variation** between the keypoints of the image.<br>
**Solution:**
- We can eleminate this problem by **croping out the image of a peron** using a CNN architecture like Yolo or anything that could detect the bounding boxes of a person.
- This image then can be fed to the openpose model to estimate keypoints for both the sources.<br>
**Scope of improvement:**
- The accuracy of the model for keypoint prediction can be increased by taking a much powerful pretrained model architecture than mobilenet.